# DataClass Review
참고 사이트 : https://realpython.com/python-data-classes/

In [1]:
from dataclasses import dataclass

@dataclass
class DataClassCard:
    rank: str
    suit: str

- 데이터를 위해 만들어진 class이기는 하지만 딱히 규제는 없음

- 선언, 출력, 값 비교 모두 가능

In [2]:
queen_of_hearts = DataClassCard('Q','Hearts')
queen_of_hearts.rank

'Q'

In [3]:
queen_of_hearts

DataClassCard(rank='Q', suit='Hearts')

In [4]:
queen_of_hearts == DataClassCard('Q','Hearts')

True

- 기본 클라스의 모양에 비해 매우 간결함

In [5]:
class RegularCard:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

- 그리고 기본 클라스의 경우 출력값의 출력과 값 비교가 안됨

In [6]:
queen_of_hearts = RegularCard('Q', 'Hearts')
queen_of_hearts

In [7]:
queen_of_hearts == RegularCard('Q', 'Hearts')

False

In [8]:
class RegularCard:
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit

    def __repr__(self):
        return (f'{self.__class__.__name__}'
                f'(rank={self.rank!r}, suit={self.suit!r})')

    def __eq__(self, other):
        if other.__class__ is not self.__class__:
            return NotImplemented
        return (self.rank, self.suit) == (other.rank, other.suit)

In [9]:
queen_of_hearts = RegularCard('Q', 'Hearts')
queen_of_hearts

RegularCard(rank='Q', suit='Hearts')

In [10]:
queen_of_hearts == RegularCard('Q', 'Hearts')

True

--------------
# 1. Alterantives to Data Classes

In [11]:
queen_of_hearts_tuple = ('Q', 'Hearts')
queen_of_hearts_dict = {'rank': 'Q', 'suit': 'Hearts'}

문제점
- queen_of_hearts_... 가 카드를 의미함을 기억해야한다 
- 튜플의 경우 ('Spades','A')로 입력해도 이해할 수 있는 에러가 나오지 않는다. 
- 사전의 경우 {'value': 'A', 'suit': 'Spades'} 는 문제가 생긴다

- 또한 사용법이 이상적이지 않다


In [12]:
queen_of_hearts_tuple[0]

'Q'

In [13]:
queen_of_hearts_dict['suit']

'Hearts'

In [14]:
from collections import namedtuple

NamedTupleCard = namedtuple('NamedTupleCard', ['rank', 'suit'])

In [15]:
queen_of_hearts = NamedTupleCard('Q', 'Hearts')
queen_of_hearts.rank

'Q'

In [16]:
queen_of_hearts

NamedTupleCard(rank='Q', suit='Hearts')

In [17]:
queen_of_hearts == NamedTupleCard('Q', 'Hearts')

True

In [18]:
# 이게 문제점!! 
queen_of_hearts == ('Q', 'Hearts')

True

In [19]:
## 변수의 타입을 정확하게 인지하게 힘들게 됨! 
Person = namedtuple('Person', ['first_initial', 'last_name'])
ace_of_spades = NamedTupleCard('A', 'Spades')
ace_of_spades == Person('A', 'Spades')

True

In [20]:
# 또한 값 변경이 안됨
card = NamedTupleCard('7', 'Diamonds')
card.rank = '9'

AttributeError: can't set attribute

In [21]:
import attr

@attr.s
class AttrsCard:
    rank = attr.ib()
    suit = attr.ib()

- 장점: 똑같이 작동하고 Python2.7부터 사용됨
- 단점: 외부 라이브러리임 (환경이 바뀔 때마다 새로 설치 필요)

--------------
# 2. Basic Data Classes

In [22]:
@dataclass
class Position:
    name: str
    lon: float
    lat: float

In [23]:
pos = Position('Oslo', 10.8, 59.9)
print(pos)

Position(name='Oslo', lon=10.8, lat=59.9)


In [24]:
pos.lat

59.9

In [25]:
print(f'{pos.name} is a t {pos.lat}°N, {pos.lon}°E')

Oslo is a t 59.9°N, 10.8°E


### 이렇게 사용도 가능!! (마치 튜플처럼)

In [26]:
from dataclasses import make_dataclass

Position = make_dataclass('Position', ['name', 'lat', 'lon'])

In [27]:
pos = Position('Oslo', 10.8, 59.9)
print(pos)

Position(name='Oslo', lat=10.8, lon=59.9)


In [28]:
## 이렇게 초기값 선언도 가능
from dataclasses import dataclass

@dataclass
class Position:
    name: str
    lon: float = 0.0
    lat: float = 0.0

### 초기에 선언된 내용 변경도 가능

In [29]:
Position('Null Island')

Position(name='Null Island', lon=0.0, lat=0.0)

In [30]:
Position('Greenwich', lat=51.8)

Position(name='Greenwich', lon=0.0, lat=51.8)

In [31]:
Position('Vancouver', -123.1, 49.3)

Position(name='Vancouver', lon=-123.1, lat=49.3)

In [32]:
## 사용할 내부의 데이터에 타입을 미리 선언하고 싶지 않을 때
from dataclasses import dataclass
from typing import Any

@dataclass
class WithoutExplicitTypes:
    name: Any
    value: Any = 42

In [33]:
## 그치만 미리 선언을 했던 데이터의 경우에도 꼭 그 데이터 타입을 사용할 필요는 없다.
Position(3.14, 'pi day', 2018)

Position(name=3.14, lon='pi day', lat=2018)

In [34]:
## 그냥 class와 마찬가지로 함수를 선언 할 수 있다

from dataclasses import dataclass
from math import asin, cos, radians, sin, sqrt

@dataclass
class Position:
    name: str
    lon: float = 0.0
    lat: float = 0.0

    def distance_to(self, other):
        r = 6371  # Earth radius in kilometers
        lam_1, lam_2 = radians(self.lon), radians(other.lon)
        phi_1, phi_2 = radians(self.lat), radians(other.lat)
        h = (sin((phi_2 - phi_1) / 2)**2
             + cos(phi_1) * cos(phi_2) * sin((lam_2 - lam_1) / 2)**2)
        return 2 * r * asin(sqrt(h))

In [35]:
oslo = Position('Oslo', 10.8, 59.9)
vancouver = Position('Vancouver', -123.1, 49.3)
oslo.distance_to(vancouver)

7181.784122942117

----------------------
# 3. More Flexible Data Classes

In [36]:
from dataclasses import dataclass
from typing import List

@dataclass
class PlayingCard:
    rank: str
    suit: str

@dataclass
class Deck:
    cards: List[PlayingCard]

In [37]:
queen_of_hearts = PlayingCard('Q', 'Hearts')
ace_of_spades = PlayingCard('A', 'Spades')
two_cards = Deck([queen_of_hearts, ace_of_spades])

In [38]:
two_cards

Deck(cards=[PlayingCard(rank='Q', suit='Hearts'), PlayingCard(rank='A', suit='Spades')])

In [39]:
RANKS = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
SUITS = '♣ ♢ ♡ ♠'.split()

def make_french_deck():
    return [PlayingCard(r, s) for s in SUITS for r in RANKS]

In [40]:
make_french_deck()

[PlayingCard(rank='2', suit='♣'),
 PlayingCard(rank='3', suit='♣'),
 PlayingCard(rank='4', suit='♣'),
 PlayingCard(rank='5', suit='♣'),
 PlayingCard(rank='6', suit='♣'),
 PlayingCard(rank='7', suit='♣'),
 PlayingCard(rank='8', suit='♣'),
 PlayingCard(rank='9', suit='♣'),
 PlayingCard(rank='10', suit='♣'),
 PlayingCard(rank='J', suit='♣'),
 PlayingCard(rank='Q', suit='♣'),
 PlayingCard(rank='K', suit='♣'),
 PlayingCard(rank='A', suit='♣'),
 PlayingCard(rank='2', suit='♢'),
 PlayingCard(rank='3', suit='♢'),
 PlayingCard(rank='4', suit='♢'),
 PlayingCard(rank='5', suit='♢'),
 PlayingCard(rank='6', suit='♢'),
 PlayingCard(rank='7', suit='♢'),
 PlayingCard(rank='8', suit='♢'),
 PlayingCard(rank='9', suit='♢'),
 PlayingCard(rank='10', suit='♢'),
 PlayingCard(rank='J', suit='♢'),
 PlayingCard(rank='Q', suit='♢'),
 PlayingCard(rank='K', suit='♢'),
 PlayingCard(rank='A', suit='♢'),
 PlayingCard(rank='2', suit='♡'),
 PlayingCard(rank='3', suit='♡'),
 PlayingCard(rank='4', suit='♡'),
 PlayingCard

### 이렇게 사용하면 에러남!! 가장 파이썬의 anti-pattern임 (using mutable default arguments)

In [41]:
from dataclasses import dataclass
from typing import List

@dataclass
class Deck:  # Will NOT work
    cards: List[PlayingCard] = make_french_deck()

ValueError: mutable default <class 'list'> for field cards is not allowed: use default_factory

## < Mutable 객체 > 
- list, dict, set, byte array, user-defined classes 등
- 함수에 paramter default로 넣게되면 변수가 초기화 되지 않고 누적됨

## < Immutable 객체 >
- int, float, decimal, complex, bool, string, tuple, range, frozenset, bytes 등

## The field() specifier is used to customize each field of a data class individually.
- default: Default value of the field
- default_factory: Function that returns the initial value of the field
- init: Use field in .__init__() method? (Default is True.)
- repr: Use field in repr of the object? (Default is True.)
- compare: Include the field in comparisons? (Default is True.)
- hash: Include the field when calculating hash()? (Default is to use the same as for compare.)
- metadata: A mapping with information about the field

In [47]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class Deck:
    cards: List[PlayingCard] = field(default_factory=make_french_deck)

In [48]:
Deck()

Deck(cards=[PlayingCard(rank='2', suit='♣'), PlayingCard(rank='3', suit='♣'), PlayingCard(rank='4', suit='♣'), PlayingCard(rank='5', suit='♣'), PlayingCard(rank='6', suit='♣'), PlayingCard(rank='7', suit='♣'), PlayingCard(rank='8', suit='♣'), PlayingCard(rank='9', suit='♣'), PlayingCard(rank='10', suit='♣'), PlayingCard(rank='J', suit='♣'), PlayingCard(rank='Q', suit='♣'), PlayingCard(rank='K', suit='♣'), PlayingCard(rank='A', suit='♣'), PlayingCard(rank='2', suit='♢'), PlayingCard(rank='3', suit='♢'), PlayingCard(rank='4', suit='♢'), PlayingCard(rank='5', suit='♢'), PlayingCard(rank='6', suit='♢'), PlayingCard(rank='7', suit='♢'), PlayingCard(rank='8', suit='♢'), PlayingCard(rank='9', suit='♢'), PlayingCard(rank='10', suit='♢'), PlayingCard(rank='J', suit='♢'), PlayingCard(rank='Q', suit='♢'), PlayingCard(rank='K', suit='♢'), PlayingCard(rank='A', suit='♢'), PlayingCard(rank='2', suit='♡'), PlayingCard(rank='3', suit='♡'), PlayingCard(rank='4', suit='♡'), PlayingCard(rank='5', suit='♡

In [49]:
from dataclasses import fields
@dataclass
class Position:
    name: str
    lon: float = field(default=0.0, metadata={'unit': 'degrees'})
    lat: float = field(default=0.0, metadata={'unit': 'degrees'})

In [50]:
fields(Position)

(Field(name='name',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x000001F358C81888>,default_factory=<dataclasses._MISSING_TYPE object at 0x000001F358C81888>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD),
 Field(name='lon',type=<class 'float'>,default=0.0,default_factory=<dataclasses._MISSING_TYPE object at 0x000001F358C81888>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'unit': 'degrees'}),_field_type=_FIELD),
 Field(name='lat',type=<class 'float'>,default=0.0,default_factory=<dataclasses._MISSING_TYPE object at 0x000001F358C81888>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({'unit': 'degrees'}),_field_type=_FIELD))

In [51]:
lat_unit = fields(Position)[2].metadata['unit']
lat_unit

'degrees'

In [52]:
from dataclasses import dataclass

@dataclass
class PlayingCard:
    rank: str
    suit: str

    def __str__(self):
        return f'{self.suit}{self.rank}'

In [53]:
ace_of_spades = PlayingCard('A', '♠')
ace_of_spades

PlayingCard(rank='A', suit='♠')

In [54]:
print(ace_of_spades)

♠A


In [55]:
print(Deck())

Deck(cards=[PlayingCard(rank='2', suit='♣'), PlayingCard(rank='3', suit='♣'), PlayingCard(rank='4', suit='♣'), PlayingCard(rank='5', suit='♣'), PlayingCard(rank='6', suit='♣'), PlayingCard(rank='7', suit='♣'), PlayingCard(rank='8', suit='♣'), PlayingCard(rank='9', suit='♣'), PlayingCard(rank='10', suit='♣'), PlayingCard(rank='J', suit='♣'), PlayingCard(rank='Q', suit='♣'), PlayingCard(rank='K', suit='♣'), PlayingCard(rank='A', suit='♣'), PlayingCard(rank='2', suit='♢'), PlayingCard(rank='3', suit='♢'), PlayingCard(rank='4', suit='♢'), PlayingCard(rank='5', suit='♢'), PlayingCard(rank='6', suit='♢'), PlayingCard(rank='7', suit='♢'), PlayingCard(rank='8', suit='♢'), PlayingCard(rank='9', suit='♢'), PlayingCard(rank='10', suit='♢'), PlayingCard(rank='J', suit='♢'), PlayingCard(rank='Q', suit='♢'), PlayingCard(rank='K', suit='♢'), PlayingCard(rank='A', suit='♢'), PlayingCard(rank='2', suit='♡'), PlayingCard(rank='3', suit='♡'), PlayingCard(rank='4', suit='♡'), PlayingCard(rank='5', suit='♡

In [56]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class Deck:
    cards: List[PlayingCard] = field(default_factory=make_french_deck)

    def __repr__(self):
        cards = ', '.join(f'{c!s}' for c in self.cards)
        return f'{self.__class__.__name__}({cards})'

In [57]:
Deck()

Deck(♣2, ♣3, ♣4, ♣5, ♣6, ♣7, ♣8, ♣9, ♣10, ♣J, ♣Q, ♣K, ♣A, ♢2, ♢3, ♢4, ♢5, ♢6, ♢7, ♢8, ♢9, ♢10, ♢J, ♢Q, ♢K, ♢A, ♡2, ♡3, ♡4, ♡5, ♡6, ♡7, ♡8, ♡9, ♡10, ♡J, ♡Q, ♡K, ♡A, ♠2, ♠3, ♠4, ♠5, ♠6, ♠7, ♠8, ♠9, ♠10, ♠J, ♠Q, ♠K, ♠A)

In [58]:
queen_of_hearts = PlayingCard('Q', '♡')
ace_of_spades = PlayingCard('A', '♠')
ace_of_spades > queen_of_hearts

TypeError: '>' not supported between instances of 'PlayingCard' and 'PlayingCard'

In [59]:
@dataclass(order=True)
class PlayingCard:
    rank: str
    suit: str

    def __str__(self):
        return f'{self.suit}{self.rank}'

In [60]:
queen_of_hearts = PlayingCard('Q', '♡')
ace_of_spades = PlayingCard('A', '♠')
ace_of_spades > queen_of_hearts

False

- init: Add .__init__() method? (Default is True.)
- repr: Add .__repr__() method? (Default is True.)
- eq: Add .__eq__() method? (Default is True.)
- order: Add ordering methods? (Default is False.)
- unsafe_hash: Force the addition of a .__hash__() method? (Default is False.)
- frozen: If True, assigning to fields raise an exception. (Default is False.)

In [61]:
from dataclasses import dataclass, field

RANKS = '2 3 4 5 6 7 8 9 10 J Q K A'.split()
SUITS = '♣ ♢ ♡ ♠'.split()

@dataclass(order=True)
class PlayingCard:
    sort_index: int = field(init=False, repr=False)
    rank: str
    suit: str

    def __post_init__(self):
        self.sort_index = (RANKS.index(self.rank) * len(SUITS)
                           + SUITS.index(self.suit))

    def __str__(self):
        return f'{self.suit}{self.rank}'

In [62]:
queen_of_hearts = PlayingCard('Q', '♡')
ace_of_spades = PlayingCard('A', '♠')
ace_of_spades > queen_of_hearts

True

In [63]:
Deck(sorted(make_french_deck()))

Deck(♣2, ♢2, ♡2, ♠2, ♣3, ♢3, ♡3, ♠3, ♣4, ♢4, ♡4, ♠4, ♣5, ♢5, ♡5, ♠5, ♣6, ♢6, ♡6, ♠6, ♣7, ♢7, ♡7, ♠7, ♣8, ♢8, ♡8, ♠8, ♣9, ♢9, ♡9, ♠9, ♣10, ♢10, ♡10, ♠10, ♣J, ♢J, ♡J, ♠J, ♣Q, ♢Q, ♡Q, ♠Q, ♣K, ♢K, ♡K, ♠K, ♣A, ♢A, ♡A, ♠A)

In [64]:
from random import sample
print(Deck(sample(make_french_deck(), k=10)))

Deck(♣4, ♠J, ♢5, ♢9, ♢Q, ♡3, ♠4, ♣7, ♣10, ♡A)


------------------
# 4. Immutable Data Classes

In [66]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Position:
    name: str
    lon: float = 0.0
    lat: float = 0.0

In [67]:
pos = Position('Oslo', 10.8, 59.9)
pos.name

'Oslo'

In [68]:
pos.name = 'Stockholm'

FrozenInstanceError: cannot assign to field 'name'

In [69]:
from dataclasses import dataclass
from typing import List

@dataclass(frozen=True)
class ImmutableCard:
    rank: str
    suit: str

@dataclass(frozen=True)
class ImmutableDeck:
    cards: List[ImmutableCard]

In [70]:
queen_of_hearts = ImmutableCard('Q', '♡')
ace_of_spades = ImmutableCard('A', '♠')
deck = ImmutableDeck([queen_of_hearts, ace_of_spades])
deck

ImmutableDeck(cards=[ImmutableCard(rank='Q', suit='♡'), ImmutableCard(rank='A', suit='♠')])

In [71]:
deck.cards[0] = ImmutableCard('7', '♢')
deck

ImmutableDeck(cards=[ImmutableCard(rank='7', suit='♢'), ImmutableCard(rank='A', suit='♠')])

## 바뀜!!! Class 내부에 선언된 데이터의 타입이 mutable 하기 때문에 (immutable하게 만드려면 list 대신 tuple 사용)

----------
# 5. Inheritance

In [72]:
from dataclasses import dataclass

@dataclass
class Position:
    name: str
    lon: float
    lat: float

@dataclass
class Capital(Position):
    country: str

In [73]:
Capital('Oslo', 10.8, 59.9, 'Norway')

Capital(name='Oslo', lon=10.8, lat=59.9, country='Norway')

In [74]:
from dataclasses import dataclass

@dataclass
class Position:
    name: str
    lon: float = 0.0
    lat: float = 0.0

@dataclass
class Capital(Position):
    country: str = 'Unknown'
    lat: float = 40.0

In [75]:
 Capital('Madrid', country='Spain')

Capital(name='Madrid', lon=0.0, lat=40.0, country='Spain')

------------
# 6. Optimizing Data Classes

In [76]:
from dataclasses import dataclass

@dataclass
class SimplePosition:
    name: str
    lon: float
    lat: float

@dataclass
class SlotPosition:
    __slots__ = ['name', 'lon', 'lat']
    name: str
    lon: float
    lat: float

In [78]:
from pympler import asizeof
simple = SimplePosition('London', -0.1, 51.5)
slot = SlotPosition('Madrid', -3.7, 40.4)
asizeof.asizesof(simple, slot)

(440, 168)

In [79]:
from timeit import timeit
timeit('slot.name', setup="slot=SlotPosition('Oslo', 10.8, 59.9)", globals=globals())


0.021591800000351213

In [80]:
timeit('simple.name', setup="simple=SimplePosition('Oslo', 10.8, 59.9)", globals=globals())

0.02032710000003135